# Training an Iris classifier using TensorFlow

In this notebook, we'll show how to train a classifier trained on the [iris data set](https://archive.ics.uci.edu/ml/datasets/iris) using TensorFlow.

## Install Dependencies
First, we'll install our dependencies:

In [0]:
pip install tensorflow==1.14.* scikit-learn==0.21.* boto3==1.*

## Load the data
We can use scikit-learn to load the Iris dataset:

In [0]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

## Train the model
We'll use TensorFlow's [`DNNClassifier`](https://www.tensorflow.org/versions/r1.14/api_docs/python/tf/estimator/DNNClassifier) to train the model:

In [0]:
import tensorflow as tf

feature_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]


def train_input_fn(features, labels, batch_size):
    irises = {}

    for i, feature_name in enumerate(feature_names):
        irises[feature_name] = features[:, i]
        
    dataset = tf.data.Dataset.from_tensor_slices((irises, labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    return dataset


def eval_input_fn(features, labels, batch_size):
    irises = {}
    for i, feature_name in enumerate(feature_names):
        irises[feature_name] = features[:, i]

    if labels is None:
        inputs = irises
    else:
        inputs = (irises, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)

    return dataset


feature_columns = [tf.feature_column.numeric_column(feature_name) for feature_name in feature_names]

classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
)

classifier.train(input_fn=lambda: train_input_fn(X_train, y_train, 100), steps=1000)

eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(X_test, y_test, 100))

print("\nTest set accuracy: {accuracy:0.3f}\n".format(**eval_result))  # Accuracy should be > 90%

## Export the model
Now we can export the model using [`Estimator.export_saved_model`](https://www.tensorflow.org/versions/r1.14/api_docs/python/tf/estimator/Estimator#export_saved_model):

In [0]:
def json_serving_input_fn():
    placeholders = {}
    features = {}
    for feature_name in feature_names:
        placeholders[feature_name] = tf.placeholder(shape=[None], dtype=tf.float64, name=feature_name)
        features[feature_name] = tf.expand_dims(placeholders[feature_name], -1)
    
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors=placeholders)


classifier.export_saved_model("export", json_serving_input_fn)

## Upload the model to AWS

Cortex loads models from AWS, so we need to upload the exported model.

Set these variables to configure your AWS credentials and model upload path:

In [0]:
AWS_ACCESS_KEY_ID = "" #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = "" #@param {type:"string"}
S3_UPLOAD_PATH = "s3://my-bucket/iris-classifier/tensorflow" #@param {type:"string"}

import sys
import re

if AWS_ACCESS_KEY_ID == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_ACCESS_KEY_ID"), file=sys.stderr)

elif AWS_SECRET_ACCESS_KEY == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_SECRET_ACCESS_KEY"), file=sys.stderr)

else:
    try:
        bucket, key = re.match("s3://(.+?)/(.+)", S3_UPLOAD_PATH).groups()
    except:
        print("\033[91m{}\033[00m".format("ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), file=sys.stderr)

Upload the model to S3:

In [0]:
import os
import boto3

s3 = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

for dirpath, _, filenames in os.walk("export"):
    for filename in filenames:
        filepath = os.path.join(dirpath, filename)
        filekey = os.path.join(key, filepath[len("export/"):])
        print("Uploading s3://{}/{}...".format(bucket, filekey), end = '')
        s3.upload_file(filepath, bucket, filekey)
        print(" ✓")
print("\nUploaded model export directory to " + S3_UPLOAD_PATH)